# 데이터 분할
머신러닝 모델을 개발하고 평가하는 과정에서 ```데이터 분할(Data Splitting)```은 매우 중요하고 기본적인 단계입니다. 데이터를 적절하게 분할하지 않으면 모델의 성능을 과대평가하거나(과적합, Overfitting), 실제 새로운 데이터에 대한 예측 능력을 제대로 파악하기 어렵습니다.

1. 모델 성능의 일반화(Generalization) 능력 평가
2. 과적합(Overfitting) 방지 및 탐지
3. 하이퍼파라미터 튜닝(Hyperparameter Tuning)

In [1]:
import pandas as pd
fish = pd.read_csv("Data/fish2.csv")
fish.head()

,Species,Weight,Length,Diagonal,Height,Width
0,Bream,242.0,25.4,30.0,11.5200,4.0200
1,Bream,290.0,26.3,31.2,12.4800,4.3056
2,Bream,340.0,26.5,31.1,12.3778,4.6961
3,Bream,363.0,29.0,33.5,12.7300,4.4555
4,Bream,430.0,29.0,34.0,12.4440,5.1340


In [2]:
fish.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 159 entries, 0 to 158
Data columns (total 6 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   Species   159 non-null    object 
 1   Weight    159 non-null    float64
 2   Length    159 non-null    float64
 3   Diagonal  159 non-null    float64
 4   Height    159 non-null    float64
 5   Width     159 non-null    float64
dtypes: float64(5), object(1)
memory usage: 7.6+ KB


In [4]:
X = fish.drop("Species", axis = 1).values
Y = fish['Species'].values

## 홀드아웃 방식 (Hold-out Method)
전체 데이터를 단순히 훈련 세트와 테스트 세트 (또는 훈련/검증/테스트 세트)로 한 번만 나누는 가장 간단한 방법입니다.

In [5]:
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import StandardScaler

train_x, test_x, train_y, test_y = train_test_split(X, Y, test_size = 0.2)

ss = StandardScaler()
train_scaled = ss.fit_transform(train_x)
test_scaled = ss.transform(test_x)

In [6]:
model = KNeighborsClassifier()
model.fit(train_scaled, train_y)
model.score(test_scaled, test_y)

0.65625

## K-겹 교차 검증 (K-Fold Cross-Validation)
데이터를 K개의 동일한 크기의 부분집합(폴드, Fold)으로 나눕니다. 총 K번의 반복을 수행하며, 매번 다른 폴드를 ```테스트 세트(또는 검증 세트)```로 사용하고 나머지 K-1개의 폴드를 훈련 세트로 사용합니다. K번의 평가 결과를 평균내어 모델의 최종 성능을 추정합니다.

In [14]:
from sklearn.model_selection import KFold, cross_val_score

kfold = KFold(n_splits=5, shuffle=True)        # 등분을 5개로 나눔

model = KNeighborsClassifier()
# scores = cross_val_score(model, X, Y, cv = kfold)        # 5개의 성능 
scores = cross_val_score(model, train_scaled, train_y, cv = kfold)        # 표준화된 데이터로 성능평가도 가능
scores.mean()         # 성능의 평균

0.8119999999999999

## 계층별 K-겹 교차 검증 (Stratified K-Fold Cross-Validation):
K-겹 교차 검증과 유사하지만, 각 폴드에 클래스(타겟)의 분포가 원본 데이터와 동일하게 유지되도록 합니다. 불균형 데이터셋(Imbalanced Dataset, 클래스 비율이 매우 다른 데이터셋)에서 특히 중요합니다.

In [11]:
from sklearn.model_selection import StratifiedKFold

kfold = StratifiedKFold(n_splits=5, shuffle=True)     # 섞긴섞는데 생선들의 비율을 맞춰서 섞겠다

scores = cross_val_score(model, X, Y, cv = kfold)        # 5개의 성능 
scores.mean()         # 성능의 평균

0.5725806451612903

## Leave-One-Out 교차 검증 (LOOCV):
K-겹 교차 검증의 특별한 경우로, K를 데이터 포인트의 수와 같게 설정합니다. (즉, 하나의 데이터 포인트를 테스트 세트로 사용하고 나머지를 학습 세트로 사용). 데이터가 매우 적을 때 사용할 수 있지만, 계산 비용이 매우 큽니다.

In [13]:
from sklearn.model_selection import LeaveOneOut

loo = LeaveOneOut()        # 들어온 데이터 개수만큼 등분

scores = cross_val_score(model, X, Y, cv = loo)        # 맞히면1, 못맞히면 0으로
scores.mean()         # 성능의 평균

0.5660377358490566